In [1]:
import torch 
import os 
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

%cd ../

/workspace/Walk_Video_PyTorch/project


In [2]:
from  models.pytorchvideo_models import WalkVideoClassificationLightningModule
from pytorch_lightning import Trainer

/usr/lib/python3.9/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [3]:
from main import get_parameters

opt, unknown = get_parameters()
opt.num_workers = 2
opt.batch_size = 16
opt.gpu_num = 0


In [4]:
model = WalkVideoClassificationLightningModule(opt)
# model = model.load_from_checkpoint("/workspace/Walk_Video_PyTorch/logs/resnet/0603/checkpoints/epoch=99-step=1800.ckpt")
model.eval()

checkpoint = torch.load("/workspace/Walk_Video_PyTorch/logs/resnet/0708_resnet_depth50/checkpoints/epoch=58-step=4307.ckpt")

model.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [5]:
from dataloader.data_loader import WalkDataModule
from pytorch_lightning import loggers as pl_loggers 

# load test dataset 
module = WalkDataModule(opt)
module.setup()
test_data = module.test_dataloader()

# for the tensorboard
tb_logger = pl_loggers.TensorBoardLogger(save_dir="/workspace/Walk_Video_PyTorch/project/tests/logs", name=opt.model, version=opt.version)


trainer = Trainer(
    accelerator="auto",
    devices=1,
    gpus=opt.gpu_num,
    logger=tb_logger,
)

results = trainer.test(dataloaders=test_data, model=model)


/usr/local/lib/python3.9/dist-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9689922332763672     │
│     test_acc_metrice      │    0.9689922332763672     │
│         test_loss         │    0.11398909986019135    │
└───────────────────────────┴───────────────────────────┘

In [6]:
input_data = next(iter(test_data))


In [7]:
input_data['video'].shape

input_data["video_name"]

['20180723_1_ASD_lat__V1-0074.mp4',
 '20180723_1_ASD_lat__V1-0043.mp4',
 '20180521_2_ASD_lat__V1-0073.mp4',
 '20180109_LCS_lat_V1-0052.mp4',
 '20160927_DHS_lat_V1-0004.mp4',
 '20171127_DHS_lat_V1-0041.mp4',
 '20170412_DHS_lat_V1-0004.mp4',
 '20190225_DHS_lat_V1-0046.mp4',
 '20190507_1_ASD_lat__V1-0018.mp4',
 '20210427_ASD_lat_V1-0002.mp4',
 '20180723_1_ASD_lat__V1-0014.mp4',
 '20170130_ASD_lat_ (12).mp4',
 '20190507_2_ASD_lat__V1-0015.mp4',
 '20210406_ASD_lat_V1-0025.mp4',
 '20191029_1_ASD_lat__V1-0005.mp4',
 '20191217_DHS_lat_V1-0013.mp4']

In [8]:
input_data['label']

tensor([0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [9]:
preds = model(input_data['video'])

In [10]:
post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=1).indices

In [11]:
pred_classes

tensor([[0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1]])

In [12]:
classname = {}

classname[0] = 'asd'
classname[1] = 'asd_not'

In [13]:
real_calss = []

for i in input_data['label'].tolist():
    real_calss.append(classname[i])

In [14]:
real_calss

['asd',
 'asd',
 'asd',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd_not']

In [15]:
# pred_class_names = []
# for num in range(opt._BATCH_SIZE):
#     for i in pred_classes[i]:
#         pred_class_names.append(classname[int(i)])


pred_class_names = [classname[int(i)] for i in pred_classes]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print("real label: %s" % ",".join(real_calss))

Predicted labels: asd, asd, asd, asd_not, asd_not, asd_not, asd_not, asd_not, asd, asd, asd, asd, asd, asd, asd, asd_not
real label: asd,asd,asd,asd_not,asd_not,asd_not,asd_not,asd_not,asd,asd,asd,asd,asd,asd,asd,asd_not


In [16]:
pred_class_names == real_calss

True

In [17]:
result = []

for i in range(len(real_calss)):
    if pred_class_names[i] == real_calss[i]:
        result.append("true")
    else:
        result.append("fale")

result

['true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true']

: 